In [ ]:
import pandas as pd
from obspy.clients.fdsn import Client
from glob import glob
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [ ]:
# Read in stations for which we have station csv information for
df = pd.read_csv("Paper Zero - Station Information - Blad1-3.csv")
lat = df['latitude'].values
lon = df['longitude'].values

In [ ]:
# Prepare FDSN clients for getting lat-lon information for non-lockdown stations
all_clients = {}
clients = ["GFZ", "IRIS", "RASPISHAKE", "KNMI",  "INGV", "BGR", "ETH", "GEONET", "KOERI", "RESIF", "LMU", "NCEDC",
           "NIEP", "NOA", "ODC",  "SCEDC", "TEXNET", "USP", "http://eida.geo.uib.no",
           "http://eida.bgs.ac.uk", "http://ws.icgc.cat/", "http://ceida.ipma.pt"]
for c in clients:
    print(c)
    try:
        all_clients[c] = Client(c)
    except Exception as e:
        print(e)
        continue

In [ ]:
# Now get the lat-lons for stations without lockdowns

lats_noLD = []
lons_noLD = []

df_no_ld = pd.read_csv("Stations with no Lockdown - Blad1-2.csv")
for idx, r in df_no_ld.iterrows():
    net = r["Station_Code"].split(".")[0]
    sta = r["Station_Code"].split(".")[1]
    print(net, sta)
    for client_name in all_clients.keys():
        client = all_clients[client_name]
        try:
            inv = client.get_stations(network=net, station=sta)
        except Exception as e:
            continue
        lats_noLD.append(inv[0][0].latitude)
        lons_noLD.append(inv[0][0].longitude)
        continue

In [ ]:
# Find the lat lons of csv files in the directory for which lat,lon information has not been entered
# into the station information spreadsheet

# Read in inviduual CSV files to get NET.STA codes
fns = glob("PaperZero_RMS/*.csv")
# Find latitude and longitude for each station. Use ad-hoc list based on ISC station registries for which some
# stations cannot be queried using FDSNWS
extra_stations = {"EFIU": {"lat": 37.78960, "lon": 15.21030},
                  "BER": {"lat": 60.38381, "lon": 5.33389},
                  "BTMR": {"lat": 44.43700, "lon": 26.10670},
                  "MNH": {"lat": 48.15000, "lon": 11.60000},
                  "THE": {"lat": 40.63220, "lon": 22.96500},
                  "GGERF": {"lat": 55.841139, "lon": -4.221389}}
sta_lats = []
sta_lons = []
sta_codes = []
for sta_csv in fns:
    net, sta, loc, cha, _ = sta_csv.split("/")[1].split(".")
    if sta in extra_stations.keys():
        sta_codes.append("{:}.{:}".format(net, sta))
        sta_lats.append(extra_stations[sta]["lat"])
        sta_lons.append(extra_stations[sta]["lon"])
    else:
        if net == "AM":
            cli = "RASPISHAKE"
        elif net == "CH":
            cli = "ETH"
        elif net == "NC":
            cli = "NCEDC"
        elif net in ["FR", "QM"]:
            cli = "RESIF"
        elif net in ["CQ", "HL"]:
            cli = "NOA"
        elif net in ["GE", "KV"]:
            cli = "GFZ"
        elif net == "IV":
            cli = "INGV"
        elif net == "BW":
            cli = "LMU"
        elif net == "OE":
            cli = "ORFEUS"
        elif net == "KO":
            cli = "KOERI"
        elif net in ["MD", "RO"]:
            cli = "NIEP"
        elif net == "NZ":
            cli = "GEONET"
        elif net == "NL":
            cli = "KNMI"
        elif net == "YS":
            #cli = "http://eida.ictja.csic.es"
            print("Could not get data for YS network")
            continue
        else:
            cli = "IRIS"
        client = Client(cli)
        try:
            inv = client.get_stations(network=net, station=sta)
        except:
            print("Could not find info for {:}.{:}".format(net, sta))
            continue
        sta_lats.append(inv[0][0].latitude)
        sta_lons.append(inv[0][0].longitude)
        sta_codes.append("{:}.{:}".format(net, sta))
# Merge latitudes and longitudes into existing dataframe
d = []
a = pd.DataFrame()
a["latitude"] = sta_lats
a["longitude"] = sta_lons
a["sta_code"] = sta_codes

orig_df = pd.read_csv("Paper Zero - Station Information - Blad1.csv")
new_merged = orig_df.merge(a, left_on="Station_Code", right_on="sta_code")
new_merged.drop(["sta_code"], axis=1)
new_merged.to_csv("StationInformationLatsLons.csv")

In [ ]:
# Now make the map!
fig = plt.figure(figsize=(14,10))
map = Basemap(projection='mill',lon_0=0)
# plot coastlines, draw label meridians and parallels.
map.drawcoastlines()
map.drawparallels(np.arange(-90,90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=[0,0,0,1])
map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='gray',lake_color='aqua')
x, y = map(lon, lat)
plt.scatter(x,y,120,marker='^',color='red',alpha=0.7, zorder=10, edgecolor="k", label="Lockdown effect seen")
x_no_ld, y_no_ld = map(lons_noLD, lats_noLD)
plt.scatter(x_no_ld,y_no_ld,120,alpha=0.7, marker='^',color='white', zorder=5, edgecolor="k", label="No lockdown effect seen")
#for label, xpt, ypt in zip(sta_codes, x, y):
    #plt.text(xpt, ypt, label, zorder=20, color="white", fontsize=8)
plt.legend()
plt.show()